In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, filters, measure, morphology
from skimage.color import label2rgb
from matplotlib.patches import Ellipse

def percent_confluence(image_594,image_fitc, alpha = 1, beta = .12, pictures = True):
    """
    percent_confluence(image_594, image_fitc, alpha , beta, pictures = True)
    Takes live/dead images of biomass and quantifies the percent confluence through 
    area of cells/area of microcarriers. 
    """

    # Load live/dead photos
    dead_load = io.imread(image_594, as_gray=True)
    dead_load = np.clip(dead_load* alpha + beta, 0, 1)

    live_load = io.imread(image_fitc, as_gray = True)



    # Segmentation: Apply thresholding to microcarrier particles
    threshold = filters.threshold_otsu(dead_load)
    mc_binary = dead_load > threshold



    #Invert MC picture to isolate the background
    mc_inverted = np.invert(mc_binary)

    #Image Subtraction to remove cell floaters
    no_floaters = live_load - mc_inverted


    #Isolating cell signal 
    cell_threshold = filters.threshold_otsu(no_floaters)
    cell_signal_binary = no_floaters > cell_threshold



    #get area of mc_binary 
    mc_load = np.clip(dead_load * alpha + beta, 0, 1)

    mc_area = np.sum(mc_load)

    #get area of mc_cell_signal 
    cell_area = np.sum(cell_signal_binary)


    #divide area of cell signal/ mc binary for percent confluence 
    percent_confluence = np.round(cell_area *100 / mc_area,2)

    
    if pictures == True:
        
        print(f"Analyzing images: {os.path.basename(image_fitc)} and {os.path.basename(image_594)}")

        # Create a figure and set of subplots
        fig, axs = plt.subplots(1, 3, figsize=(20, 5)) 

        # Original image
        axs[0].imshow(live_load)
        axs[0].set_title('Original Live Image')

        # Binary image
        axs[1].imshow(mc_load)
        axs[1].set_title('MC Binary Image')

        # Eroded binary image
        axs[2].imshow(cell_signal_binary)
        axs[2].set_title('Cell Signal Binary Image')


        plt.tight_layout()
        plt.show()
        
        print(f'Percent Confluence: {percent_confluence}%')
        print('')
        print('')
        print('')
        print("******************************************************************************")
        
        return f'Percent Confluence: {percent_confluence}%'

        
def process_directory(directory_path):
    # List all files in the directory
    files = os.listdir(directory_path)
    
    # Create an empty list to store the results
    results = []
    
    # Find pairs of images that differ only by _fitc and _af594
    for file in files:
        if '_fitc' in file:
            # Construct the corresponding _af594 filename
            af594_file = file.replace('_fitc', '_af594')
            
            # Check if the corresponding _af594 file exists in the directory
            if af594_file in files:
                # Construct full file paths
                fitc_path = os.path.join(directory_path, file)
                af594_path = os.path.join(directory_path, af594_file)
                
                # Run the percent_confluence function on this pair of images
                result = percent_confluence(af594_path, fitc_path)
                
                # Append the result, and filenames to the results list
                # Extract the percentage value from the result string
                percent_value = float(result.replace('Percent Confluence: ', '').replace('%', ''))
                results.append({
                    'Image_FITC': file,
                    'Image_AF594': af594_file,
                    'Percent_Confluence': percent_value
                })
                
    # Convert the results list to a DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df